# Turtle Global Navigation

## Background

Using proprietary solid-state accelerometers and gyroscopes Turtle Global Navigation has
developed a new inertial guidance system. At each change in course the system reports the
previous azimuthal bearing $\alpha_i$ of travel from due North, and a net travel
distance along the terrestrial great circle arc $\rho_i$.

The azimuthal bearing $\alpha_i$ is reported as the shortest difference in radians from
due North:

* $0 \text{ rad}$ due North.
* $\frac{\pi}{2} \text{ rad}$ due East.
* $\pi \text{ rad}$ due South.
* $-\frac{\pi}{2} \text{ rad}$ due West.

This generates the condition $-\pi < \alpha_i \le \pi$. The travel distance
$\rho_i$ is reported in the change of radians, and is automatically reduced by the system
using modular arithmetic to find the geodesic shortest path. This generates the condition
$0 \le \rho_i \le \pi$. We can always assume these conditions will be satisfied by the input
data.

## Objective

To verify the operation of the inertial guidance system we need to reconstruct the waypoints
of the path from an initial location $\phi_0$ and $\theta_0$, indexed $0$, and a list of
bearings and distances, indexed by $1 \le i$. We are required to increment a list of
latitude $\phi_{i-1}$ and longitude $\theta_{i-1}$ waypoints with a new latitude $\phi_i$
and longitude $\theta_i$ calculated from the previous latitude $\phi_{i-1}$ and longitude
$\theta_{i-1}$ and the new bearing $\alpha_i$ and distance $\rho_i$.

All latitudes $\phi_i$ must be reported in radians from the Equator:

* $\frac{\pi}{2} \text{ rad}$ North Pole.
* $0 \text{ rad}$ Equator.
* $-\frac{\pi}{2} \text{ rad}$ South Pole.

This generates the constraint $-\frac{\pi}{2} \le \phi_i \le \frac{\pi}{2}$. Likewise all
longitudes $\theta_i$ must be reported in the shortest radians from the Prime Meridian:

* $0 \text{ rad}$ Prime Meridian.
* $\pi \text{ rad}$ International Date Line.
* Positive to the East of the Prime Meridian.
* Negative to the West of the Prime Meridian.

This generates the constraint $-\pi < \theta_i \le \pi$. To calculate the next waypoint we
will implement two
[Spherical Trigonometry](https://en.wikipedia.org/wiki/Spherical_trigonometry) calculations.

In [ ]:
# Imports go here
from math import sin, cos, acos, pi

## Update Latitude

First, we will calculate the new latitude $\phi_i$ from the previous latitude $\phi_{i-1}$
and the current bearing $\alpha_i$ and distance $\rho_i$:
$$
\phi_i = \frac{\pi}{2} - \arccos \left(
    \cos \left( \frac{\pi}{2}-\phi_{i-1} \right) \cdot
    \cos \left( \rho_i \right) +
    \sin \left( \frac{\pi}{2}-\phi_{i-1} \right) \cdot
    \sin \left( \rho_i \right) \cdot
    \cos \left( \alpha_i \right)
\right)
$$

In [ ]:
def updatelatitude(oldlatitude, bearing, distance):
    """From the previous latitude and current bearing and distance calculate the new latitude
    using Spherical Trigonometry. Returns a number:
    * `newlatitude` updated latitude."""
    return pi/2 - acos(
        (
            cos(pi/2 - oldlatitude) *
            cos(distance)
        ) + (
            sin(pi/2 - oldlatitude) *
            sin(distance) *
            cos(bearing)
        )
    )

### Unit Test Latitude Edge Cases

We can test the latitude updater to make sure it returns the correct values for selected
edge cases.

In [ ]:
# Walking East or West on the Equator should stay on the Equator
print(updatelatitude(0, pi/2, pi/4) == 0)
print(updatelatitude(0, -pi/2, pi/4) == 0)

# Walking due North or South for one eight of a circle should get to halfway to each Pole.
print(updatelatitude(0, 0, pi/4) == pi/4)
print(updatelatitude(0, pi, pi/4) == -pi/4)


## Update Longitude

Second, using the new latitude $\phi_i$ we will calculate the new longitude $\theta_i$ from
the old latitude $\phi_{i-1}$, the old longitude $\theta_{i-1}$, the current bearing
$\alpha_i$, and the current distance $\rho_i$:
$$
\theta_i = \theta_{i-1} + \operatorname{sgn}\left( \alpha_i \right) \cdot \arccos \left(
    \frac{
        \cos \left( \rho_i \right) -
        \cos \left( \frac{\pi}{2} - \phi_i \right) \cdot
        \cos \left( \frac{\pi}{2} - \phi_{i-1} \right)
    }{
        \sin \left( \frac{\pi}{2} - \phi_i \right) \cdot
        \sin \left( \frac{\pi}{2} - \phi_{i-1} \right)
    }
\right)
$$ 

In [ ]:
def updatelongitude(oldlatitude, oldlongitude, newlatitude, bearing, distance):
    """From the previous longitude, current and previous latitude and current bearing and
    distance calculate the new longitude using Spherical Trigonometry. Returns a number:
    * `newlongitude` updated longitude."""

    # Heading due North. No change in longitude.
    if bearing == 0:
        return oldlongitude
    
    # Heading due South. No change in longitude.
    if bearing == pi:
        return oldlongitude
    
    # Heading East is positive.
    easterly = 1 if bearing > 0 else -1

    # Update the longitude
    return oldlongitude + easterly * acos(
        (
            cos(distance) - (
                cos(pi/2 - newlatitude) *
                cos(pi/2 - oldlatitude)
            )
        ) / (
            sin(pi/2 - newlatitude) *
            sin(pi/2 - oldlatitude)
        )
    )

### Unit Test Longitude Edge Cases

We can test the longitude updater to make sure it returns the correct values for selected
edge cases.

In [ ]:
# Longitude does not change when walking due North or South
print(updatelongitude(pi/8, pi/16, pi/4, 0, pi/8) == pi/16)
print(updatelongitude(-pi/8, pi/16, -pi/4, pi, pi/8) == pi/16)

# Longitude changes when walking East or West on the Equator
print(updatelongitude(0, 0, 0, pi/2, pi/4) == pi/4)
print(updatelongitude(0, 0, 0, -pi/2, pi/4) == -pi/4)

## Naive Implementation

In the naive implementation we instantiate a list with the initial position and then
`append` to the list for when give a bearing and distance. This is a workable prototype
but does not separate concerns.

In [ ]:
# List of pairs of latitudes and longitudes, initialized to a starting point
waypoints = [ ( pi/4, pi/4 ) ]

# Display
print(waypoints)

In [ ]:
bearing = 5*pi/8
distance = pi/100

# Calculate new position
newlatitude = updatelatitude(
    waypoints[-1][0],
    bearing,
    distance
)
newlongitude = updatelongitude(
    waypoints[-1][0],
    waypoints[-1][1],
    newlatitude,
    bearing,
    distance
)

# Store in the list
waypoints.append(( newlatitude, newlongitude ))

# Display
print(waypoints)

In [ ]:
bearing = pi
distance = pi/10

# Calculate new position
newlatitude = updatelatitude(
    waypoints[-1][0],
    bearing,
    distance
)
newlongitude = updatelongitude(
    waypoints[-1][0],
    waypoints[-1][1],
    newlatitude,
    bearing,
    distance
)

# Store in the list
waypoints.append(( newlatitude, newlongitude ))

# Display
print(waypoints)

Notice how we have to copy the code to replicate the translation process. This is
inefficient, impractical, subject to errors, and cannot be automated.

## Closure Implementation

A more sophisticated pattern to protect the management of the list is to write a
[function closure](https://en.wikipedia.org/wiki/Closure_(computer_programming)). This is
a function that returns a function. The returned function has been initialized with the 
starting position, and will return a reference to the updated list. We can also use the
closure to protect the implementation details of the numerical updates as sub-functions.
In general functions that return executables are called *factory* functions, because they
create an object or method that can be used to do an action.

In [ ]:
def factory(latitude, longitude):
    """Initialize the position updating engine with the starting position. Returns a
    function:
    * `engine` position updater."""

    # Start our waypoints with the initial latitude and longitude.
    # This maintains the state of the path between successive calls to the
    # updating function.
    waypoints = [ ( latitude, longitude) ]

    def engine(bearing, distance):
        """Location updating function, to move on the globe from the last waypoint using the
        bearing and distance. Returns a reference to the waypoints list:
        * `waypoints` updated path."""

        # Update the path list
        newlatitude = updatelatitude(
            waypoints[-1][0],
            bearing,
            distance
        )
        newlongitude = updatelongitude(
            waypoints[-1][0],
            waypoints[-1][1],
            newlatitude,
            bearing,
            distance
        )
        waypoints.append(( newlatitude, newlongitude ))

        # Send out a reference to the paths.
        return waypoints

    # Send out the initialized updater.
    return engine

### Test the Closure

Lets put a couple of turtles in Asia and move them around.

In [ ]:
# Create some functions here.
turtleA = factory(pi/4, pi/4)
turtleB = factory(pi/4, pi/4)
print(type(turtleA))
print(type(turtleB))
print(turtleA == turtleB)
print(id(turtleA))
print(id(turtleB))

We can make multiple calls to each turtle, which independently updates each turtle's
waypoints. Notice how they move to different locations.

In [ ]:
# Lets update each one in a different order
waypointsA = turtleA(5*pi/8, pi/100)
waypointsB = turtleB(pi, pi/10)
print(type(waypointsA))
print(type(waypointsB))
print(waypointsA == waypointsB)
print(id(waypointsA))
print(id(waypointsB))
print(waypointsA)
print(waypointsB)


Note that the returned reference points to the unique list of waypoints for each turtle.

In [ ]:
# Lets update each one in a different order
waypointsC = turtleA(pi, pi/10)
waypointsD = turtleB(5*pi/8, pi/100)
print(waypointsA == waypointsC)
print(waypointsB == waypointsD)
print(id(waypointsC))
print(id(waypointsD))
print(waypointsA)
print(waypointsB)

The factory is fully reusable. We can track more than just turtles, we can track ants,
whales, lions, or tigers.

In [ ]:
moveswimmingant = factory(0, pi)
antwaypoints = moveswimmingant(-7*pi/12, pi/1000)
print(antwaypoints)

## Class Implementation

As well as the data types for strings, numbers, lists, dictionaries, and much more, we can
programmatically extend Python to have new developer defined data types. The data types
extended using the [class](https://docs.python.org/3/tutorial/classes.html) language
construct. Specific instances of the data type are instantiated by calling the class name as
a function, optionally passing in initializing values. The Python naming convention is to
capitalize the names of all classes.

### Magics

There are special attributes and methods that either come predefined or must be set when
developing a particular interface. These attributes and methods are surrounded in pairs
of underscore characters. This is only a convention to tell developers to pay attention to
a specific methods or attribute. Depending on the context a magic is either intended to be
left alone, or it requires defining. Python is permissive. The interpreter does not enforce
protection. Instead the developers have to code well behaved programs.

Affectionately known as *"dunder"* for **d**ouble **under**core. A few examples:

In [ ]:
print(__name__)
print(__IPYTHON__)
print(__doc__)
print(__vsc_ipynb_file__)

As stated before, Python is very permissive. You can both create your own *dunders* and
override existing *dunders*. However, overriding a *dunder* will break runtime
functionality.

In [ ]:
__myspecialdunder__ = 42
print(__myspecialdunder__)
__myspecialdunder__ = "forty two"
print(__myspecialdunder__)

The most common scenarios you will see using *dunders* is in `__init__.py` files or
packages. The `__name__` dunder is used to detect if the package is being run as an
independent program or being loaded into another program.

In [ ]:
if __name__ == "__main__":
    # File is being run as a stand alone program.
    print("Run as a stand alone program.")
else:
    # called from somewhere else to be loaded into a pre-existing program.
    print("Called to be loaded into a program.")

### Decorators

Python decorator are syntactic sugar for calling a function that modifies an input function.
That is a function that takes a single function as an argument and returns a function,
typically based on the input function.

A pseudo-code example propagates a fixed uncertainty in a numerical calculation:
```python
def factory(function):
    """Factory method to returns a function built from the input function."""
    def engine(input):
        """Return a range based on the input function"""
        return function(input - 0.001), function(input + 0.001)
    return engine

# Usage example of the factor decorator
@factory
def numericalmethod(input):
    """Calculate an output from the input"""
    output = exp(input) / input
    return output
```
Decorators are an important tool in developing Python classes. The `@staticmethod`
decorator is used to create a method that can be called from the class rather than an
instance of the class.

We now know enough language constructs to implement Turtle Navigation inside a class, along
with a couple of methods.

In [ ]:
from math import sin, cos, acos, pi
class Navigator:
    """Each instance holds a unique list of waypoints, initialized to different starting
    points. The instance encapsulates all the methods used to consistently maintain the
    waypoints."""

    @staticmethod
    def updatelatitude(oldlatitude, bearing, distance):
        """Static method to update from the previous latitude and current bearing and
        distance calculate the new latitude using Spherical Trigonometry. Returns a number:
        * `newlatitude` updated latitude."""
        print("Static latitude called")
        return pi/2 - acos(
            (
                cos(pi/2 - oldlatitude) *
                cos(distance)
            ) + (
                sin(pi/2 - oldlatitude) *
                sin(distance) *
                cos(bearing)
            )
        )

    @staticmethod
    def updatelongitude(oldlatitude, oldlongitude, newlatitude, bearing, distance):
        """Static method to update from the previous longitude, current and previous 
        latitude and current bearing and distance calculate the new longitude using
        Spherical Trigonometry. Returns a number:
        * `newlongitude` updated longitude."""
        print("Static longitude called")

        # Heading due North. No change in longitude.
        if bearing == 0:
            return oldlongitude
        
        # Heading due South. No change in longitude.
        if bearing == pi:
            return oldlongitude
        
        # Heading East is positive.
        easterly = 1 if bearing > 0 else -1

        # Update the longitude
        return oldlongitude + easterly * acos(
            (
                cos(distance) - (
                    cos(pi/2 - newlatitude) *
                    cos(pi/2 - oldlatitude)
                )
            ) / (
                sin(pi/2 - newlatitude) *
                sin(pi/2 - oldlatitude)
            )
        )

    def __init__(self, latitude = 0, longitude = 0):
        """Initializer method that initializes the waypoints storage to the start location.
        It is expected that you override this `__dunder__` with your constructor code.
        Returns nothing."""

        # Creates a new waypoints property containing the list of waypoints.
        self.waypoints = [ ( latitude, longitude) ]

    def move(self, bearing, distance):
        """Instance method to update the waypoints list with the new position based on the
        direction and distance traveled. Returns noting."""
        
        # Update the path list
        newlatitude = Navigator.updatelatitude(
            self.waypoints[-1][0],
            bearing,
            distance
        )
        newlongitude = Navigator.updatelongitude(
            self.waypoints[-1][0],
            self.waypoints[-1][1],
            newlatitude,
            bearing,
            distance
        )
        self.waypoints.append(( newlatitude, newlongitude ))

    def recenter(self, latitude = 0, longitude = 0):
        """"Instance method to Empty the waypoints and start and the new position. Returns
        nothing."""
        self.waypoints = [ ( latitude, longitude) ]

### Test the Class

Call the `Navigator()` constructor to create new instances.

In [ ]:
# Calling the class constructor method. Implicitly runs the __init__ method
antA = Navigator(longitude=pi/4)
print(type(antA))
print(antA)
print(antA.waypoints)
antA.move(0, pi/4)
print(antA.waypoints)
Santa = Navigator(latitude=pi/2)
print(Santa)
antB = Navigator()
print(antB.waypoints)
antC = Navigator()
print(antC.waypoints)
Santa.recenter(0, -pi/2)
print(Santa.waypoints)

### Advanced Concepts

Using the `__new__(constructor)` function in a class definition one can implement more
advance techniques like
[static](https://en.wikipedia.org/wiki/Class_%28computer_programming%29#Uninstantiable)
classes that cannot be instantiated,
[singleton](https://en.wikipedia.org/wiki/Singleton_pattern) classes, and
[interned](https://en.wikipedia.org/wiki/Interning_(computer_science)) instances.

In [ ]:
class NotConstructable:
    """"This class cannot be instantiated."""
    def __new__(constructor):
        """"Override instantiation."""
        pass

In [ ]:
x = NotConstructable()
print(type(x))
print(type(None))

Static, non-constructable, or non-instantiable classes are discouraged in Python,
particularly as containers or namespaces for methods and attributes. Instead it is strongly
recommended to import Python modules, simply `*.py` files, as the name of the file acts
as the namespace for all the definitions and declarations in the file.

Close to the singleton pattern, the instance interning pattern is an important technique to
ensure that a property or attribute of a class is unique across all the instances. This
is particularly useful for ensuring only one instance runs per processor, file, or
connection to a peripheral.

In the example below we intern by the `name` attribute to ensure there is only one instance
of the object per name. We do this by using a simple dictionary. Note the overriding of the
`__str__(self)` magic method to customize the print out of the object details. We do this
so that we can easily verify the non-duplication of names in objects.

In [ ]:
class Interned:
    """"Constrain the class to only have unique instances for each name."""

    instances = dict()
    """Unique instances of each named object."""

    def __new__(constructor, name):
        """Check to see if an instance with the same name has been created."""
        if name not in constructor.instances:
            constructor.instances[name] = object.__new__(constructor)
        return constructor.instances[name]

    def __init__(self, name):
        """Recording the objects own name."""
        self.name = name

    def __str__(self):
        """Identify the name and object."""
        return f"{self.name}: {id(self)}"


In [ ]:
x = Interned("Todd")
print(x)
y = Interned("Todd")
print(y)
print(x == y)
z = Interned("Lily")
print(z)
print(Interned.instances)

## Redistribution

Turtle Navigation requires that the inertial navigation class be redistributable. We use
Python packages to make our code redistributable. A package is simply a directory containing
a special `__init__.py` file, and other regular `*.py` files, called the modules of the
package. The `__init__.py` file defines what should occur when the package is imported into
a project. 

In [ ]:
import examplemodule
import examplepackage

Now that we have organized and packaged our class lets test importing the package in a few
different ways.

In [ ]:
import TurtleUniversal
from TurtleUniversal import CartesianNavigator, SphericalNavigator
from math import pi
print(TurtleUniversal.NAUTICALMILESPERDEGREE)
print(TurtleUniversal.Cartesian.updatelatitude)
print(TurtleUniversal.Spherical.updatelatitude)

32
<function updatelatitude at 0x000001D81CD85940>
<function updatelatitude at 0x000001D81CC47F60>


In [2]:
antA = SphericalNavigator(longitude=pi/4)
print(type(antA))
print(antA)
print(antA.waypoints)
antA.move(0, pi/4)
print(antA.waypoints)
Santa = SphericalNavigator(latitude=pi/2)
print(Santa)
antB = SphericalNavigator()
print(antB.waypoints)
antC = SphericalNavigator()
print(antC.waypoints)
Santa.recenter(0, -pi/2)
print(Santa.waypoints)
tronA = CartesianNavigator(1, 1)
print(tronA)
tronA.move(pi/8, 2)
print(tronA.waypoints)

<class 'TurtleUniversal.SphericalNavigator'>
[(0, 0)]
[(0, 0), (0.7853981633974483, 0)]
[(0, 0)]
[(0, 0)]
[(0, -1.5707963267948966)]
[(0, 0), (0.7653668647301796, 1.8477590650225735)]
